# Find interpolation points via subsampling product basis, and performing QR decomposition

In [2]:
"""Build Benzene molecule with a minimal GTO basis, usign PYSCF
"""
%load_ext autoreload
%autoreload 2

from pyscf import gto, dft
from rdkit import Chem


benzene_coordinates = """
 C         -0.65914       -1.21034        3.98683
 C          0.73798       -1.21034        4.02059
 C         -1.35771       -0.00006        3.96990
 C          1.43653       -0.00004        4.03741
 C         -0.65915        1.21024        3.98685
 C          0.73797        1.21024        4.02061
 H         -1.20447       -2.15520        3.97369
 H          1.28332       -2.15517        4.03382
 H         -2.44839       -0.00006        3.94342
 H          2.52722       -0.00004        4.06369
 H         -1.20448        2.15509        3.97373
 H          1.28330        2.15508        4.03386
"""

# Generate ball and stick data of atomic positions from SMILES
rdkit_mol = Chem.MolFromSmiles("c1ccccc1")
rdkit_mol = Chem.AddHs(rdkit_mol)

# PYSCF Molecule
mol = gto.Mole()
mol.atom = benzene_coordinates
mol.basis = 'def2-SVP'
mol.build()

# Solve SCF for restricted KS-LDA
mf = dft.RKS(mol)
mf.kernel()
# Occupied states for benzene, with this basis is 22
n_occ = 22

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
converged SCF energy = -229.930646528208


In [4]:
"""Define real-space grid
"""
from pathlib import Path

import numpy as np
from pyscf.tools import cubegen

# Root for outputs
output_root = Path('../outputs/points_via_qr')

# Grid/cube settings
nx, ny, nz = 10, 10, 10
n_total = np.prod([nx, ny, nz])

# Generate the real-space grid
cube_grid = cubegen.Cube(mol, nx, ny, nz)
grid_points = cube_grid.get_coords()
assert grid_points.shape == (n_total, 3)

In [5]:
"""Build the electron density
"""
from pathlib import Path

cube_file = output_root / 'benzene_density.cube'

# 1-electron Density matrix
dm = mf.make_rdm1()
rho = cubegen.density(mol, cube_file.as_posix(), dm, nx=nx, ny=ny, nz=nz)
rho = rho.reshape(-1)
assert rho.shape == (n_total,)

In [6]:
"""KS molecular orbitals
"""
# Generate MOs on real-space grid
wfs = np.empty(shape=(n_total, n_occ))
for i in range(n_occ):
    cube_file = output_root / f'benzene_mo{i}.cube'
    # pyscf expects a string for fname
    molecular_orbital = cubegen.orbital(mol, cube_file.as_posix(), mf.mo_coeff[:, i], nx=nx, ny=ny, nz=nz)
    wfs[:, i] = molecular_orbital.reshape(-1)


In [38]:
""" Interpolation point selection
"""
from isdf_prototypes.interpolation_points_via_qr import randomly_sample_product_matrix, interpolation_points_via_qrpivot

# Create sub-sampled pair-product Z matrix
n_interp = 100 # i.e. ~ sqrt(n_total)
print(f'Requested {n_interp} interpolation points')

z_sampled = randomly_sample_product_matrix(wfs, n_interp, random_seed=42)
assert z_sampled.shape[0] == n_total, 'Number of grid points'

# Update number of interpolation points
n_interp = z_sampled.shape[1]
print(f'Returned {n_interp} interpolation points')

# Use QR decomposition with pivoting to find interpolation points
pivot_indices = interpolation_points_via_qrpivot(z_sampled)
print("Row indices to be used as interpolation points:")
print(pivot_indices)


Requested 100 interpolation points
Returned 100 interpolation points
Rows to be used as interpolation points
[243 244 245 246 254 255 256 324 325 326 334 335 342 343 344 345 346 352
 353 354 355 356 363 364 365 366 374 375 376 423 424 425 433 434 435 436
 443 444 445 446 453 454 455 456 463 464 465 466 474 475 524 525 532 533
 534 535 536 544 545 546 553 554 555 556 563 564 565 566 567 575 623 624
 625 633 634 635 636 643 644 645 646 647 653 654 655 656 657 663 664 665
 666 673 674 675 744 745 746 754 755 756]


In [39]:
# Create pair-product Z matrix on real-space grid and export to cube (will need to use my own routine)
# Instead, generate the electron density and output this for checking the interpolation points against

# TODO
# Use pivot_indices as a mask for grid, to get the interpolation points
# Plot interpolation points over the electron density
# Plot interpolation points over |Psi_homo|^2